# CRNN OCR with Compressed Image
- 모델: CRNN
- 데이터: AI Hub 한국어 OCR 이미지 + 압축 이미지 생성기 활용
- 비교: 압축 전후 이미지에 대한 OCR 성능 (WER, Accuracy)

In [158]:
# CRNN 코드 다운로드
if not os.path.exists("crnn.pytorch"):
    !git clone https://github.com/meijieru/crnn.pytorch.git

import sys
sys.path.append("./crnn.pytorch")

In [159]:
# 필수 패키지 설치
!pip install opencv-python matplotlib editdistance jiwer pillow

Defaulting to user installation because normal site-packages is not writeable


In [160]:
# pre-trained 모델 다운로드


--2025-05-20 16:08:51--  https://www.dropbox.com/s/6j0k5g1h9g5q9we/crnn.pth?raw=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.84.18, 2620:100:6034:18::a27d:5412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.84.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/crnn.pth’

data/crnn.pth           [  <=>               ]  70.33K   227KB/s    in 0.3s    

2025-05-20 16:08:52 (227 KB/s) - ‘data/crnn.pth’ saved [72015]



# AI Hub OCR Dataset 준비


In [148]:
import os
os.chdir("/home/elicer")

In [149]:
!ls

AIHub_OCR_sample       dncnn_baseline.pth	  Original.jpeg
AIHub_OCR_sample.zip   EDSR-PyTorch		  output_compressed
BasicSR		       Low.jpeg			  Real-ESRGAN
content		       Low_test2.jpeg		  Real-ESRGAN_Dataset.ipynb
CRNN_Experiment.ipynb  Low_test.jpeg		  Real-ESRGAN.ipynb
crnn.pytorch	       Low_test_samesize.jpeg	  RFDN.ipynb
data		       my_files.zip		  Untitled.ipynb
DIV2K_train_HR.zip     Normal.jpeg		  weights
DIV2K_valid_HR.zip     Normal_test.jpeg
DnCNN		       Normal_test_samesize.jpeg


In [150]:
# 압축 해제
!unzip -o AIHub_OCR_sample.zip -d AIHub_OCR_sample

Archive:  AIHub_OCR_sample.zip
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000001.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000002.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000003.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000004.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000005.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000006.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000007.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000008.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000009.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000010.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000011.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/

  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000449.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000450.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000451.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000452.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000453.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000454.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000455.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000456.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000457.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000458.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000459.json  
  inflating: AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000460.json  
  in

  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000004.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000005.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000006.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000007.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000008.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000009.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000010.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000011.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000012.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000013.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000014.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000015.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]

  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000108.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000109.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000110.JPG  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000111.JPG  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000112.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000113.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000114.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000115.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000116.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000117.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000118.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000119.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]

  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000215.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000216.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000217.JPG  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000218.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000219.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000220.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000221.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000222.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000223.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000224.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000225.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000226.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]

  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000421.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000422.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000423.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000424.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000425.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000426.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000427.JPG  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000428.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000429.JPG  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000430.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000431.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000432.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]

  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000526.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000527.JPG  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000528.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000529.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000530.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000531.JPG  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000532.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000533.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000534.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000535.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000536.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000537.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]

  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000631.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000632.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000633.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000634.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000635.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000636.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000637.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000638.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000639.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000640.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000641.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000642.jpg  
  inflating: AIHub_OCR_sample/원천데이터/[원천]

In [151]:
# 파일 확인
!ls AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류
!ls AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류

책표지_총류_000001.json  책표지_총류_000234.json  책표지_총류_000467.json
책표지_총류_000002.json  책표지_총류_000235.json  책표지_총류_000468.json
책표지_총류_000003.json  책표지_총류_000236.json  책표지_총류_000469.json
책표지_총류_000004.json  책표지_총류_000237.json  책표지_총류_000470.json
책표지_총류_000005.json  책표지_총류_000238.json  책표지_총류_000471.json
책표지_총류_000006.json  책표지_총류_000239.json  책표지_총류_000472.json
책표지_총류_000007.json  책표지_총류_000240.json  책표지_총류_000473.json
책표지_총류_000008.json  책표지_총류_000241.json  책표지_총류_000474.json
책표지_총류_000009.json  책표지_총류_000242.json  책표지_총류_000475.json
책표지_총류_000010.json  책표지_총류_000243.json  책표지_총류_000476.json
책표지_총류_000011.json  책표지_총류_000244.json  책표지_총류_000477.json
책표지_총류_000012.json  책표지_총류_000245.json  책표지_총류_000478.json
책표지_총류_000013.json  책표지_총류_000246.json  책표지_총류_000479.json
책표지_총류_000014.json  책표지_총류_000247.json  책표지_총류_000480.json
책표지_총류_000015.json  책표지_총류_000248.json  책표지_총류_000481.json
책표지_총류_000016.json  책표지_총류_000249.json  책표지_총류_000482.json
책표지_총류_000017.json  책표지_총류_000250.json  책표지_총류_000483.js

책표지_총류_000001.jpg	책표지_총류_000234.JPG	책표지_총류_000467.jpg
책표지_총류_000002.jpg	책표지_총류_000235.jpg	책표지_총류_000468.jpg
책표지_총류_000003.jpg	책표지_총류_000236.jpg	책표지_총류_000469.jpg
책표지_총류_000004.jpg	책표지_총류_000237.jpg	책표지_총류_000470.jpg
책표지_총류_000005.jpg	책표지_총류_000238.JPG	책표지_총류_000471.jpg
책표지_총류_000006.jpg	책표지_총류_000239.jpg	책표지_총류_000472.jpg
책표지_총류_000007.jpg	책표지_총류_000240.jpg	책표지_총류_000473.jpg
책표지_총류_000008.jpg	책표지_총류_000241.jpg	책표지_총류_000474.JPG
책표지_총류_000009.jpg	책표지_총류_000242.JPG	책표지_총류_000475.jpg
책표지_총류_000010.jpg	책표지_총류_000243.jpg	책표지_총류_000476.jpg
책표지_총류_000011.jpg	책표지_총류_000244.jpg	책표지_총류_000477.JPG
책표지_총류_000012.jpg	책표지_총류_000245.jpg	책표지_총류_000478.jpg
책표지_총류_000013.jpg	책표지_총류_000246.jpg	책표지_총류_000479.jpg
책표지_총류_000014.jpg	책표지_총류_000247.jpg	책표지_총류_000480.jpg
책표지_총류_000015.jpg	책표지_총류_000248.jpg	책표지_총류_000481.jpg
책표지_총류_000016.jpg	책표지_총류_000249.jpg	책표지_총류_000482.jpg
책표지_총류_000017.jpg	책표지_총류_000250.jpg	책표지_총류_000483.jpg
책표지_총류_000018.jpg	책표지_총류_000251.jpg	책표지_총류_000484.jpg
책표지_총류_000019.jpg	책표지_총류_000

In [152]:
#GT 라벨 텍스트 추출
import json

def extract_gt_text(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # annotation id 순으로 정렬해서 text 추출
    annotations = sorted(data["annotations"], key=lambda x: x["id"])
    gt_text = " ".join([ann["text"] for ann in annotations])
    return gt_text

gt = extract_gt_text(
    "AIHub_OCR_sample/라벨링데이터/[라벨]Training/2.책표지/01.총류/책표지_총류_000001.json"
)
print("GT:", gt)


GT: 나를키우는힘 평생 독서


In [153]:
#CRNN 모델 불러오기 및 예측 함수 정의
import torch
from PIL import Image
import torchvision.transforms as transforms
import models.crnn as crnn
import utils  # crnn.pytorch/utils.py 안에 있어야 함

def load_model(model_path):
    model = crnn.CRNN(32, 1, 37, 256)  # 37은 알파벳+숫자+CTC blank 기준
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    model.eval()
    return model

def predict_text(model, image_path):
    img = Image.open(image_path).convert('L')
    transform = transforms.Compose([
        transforms.Resize((32, 100)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = transform(img).unsqueeze(0)
    converter = utils.strLabelConverter('0123456789abcdefghijklmnopqrstuvwxyz')
    preds = model(image)
    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    sim_pred = converter.decode(preds.data, raw=False)
    return sim_pred


In [156]:
!ls

AIHub_OCR_sample       dncnn_baseline.pth	  Original.jpeg
AIHub_OCR_sample.zip   EDSR-PyTorch		  output_compressed
BasicSR		       Low.jpeg			  Real-ESRGAN
content		       Low_test2.jpeg		  Real-ESRGAN_Dataset.ipynb
CRNN_Experiment.ipynb  Low_test.jpeg		  Real-ESRGAN.ipynb
crnn.pytorch	       Low_test_samesize.jpeg	  RFDN.ipynb
data		       my_files.zip		  Untitled.ipynb
DIV2K_train_HR.zip     Normal.jpeg		  weights
DIV2K_valid_HR.zip     Normal_test.jpeg
DnCNN		       Normal_test_samesize.jpeg


In [157]:
#OCR 수행 및 평가
# 모델 불러오기
model = load_model("")

# 이미지 경로
image_path = "AIHub_OCR_sample/원천데이터/[원천]Training_책표지1/01.총류/책표지_총류_000001.jpg"

# 예측 실행
pred = predict_text(model, image_path)

# 결과 출력
print("GT:", gt)
print("Pred:", pred)


UnpicklingError: invalid load key, '<'.